<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #6: Evaluate machine learning models</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: October 11, 2022</h5>

# 1. Set up and requirements

### Install and import Python packages

In [77]:
try:
  import google.colab
  import os
  IN_COLAB = True
  print("Running in Colab...")

  # Clone repo
  !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
  !pip install -q --upgrade pip
  !pip install -qr koster_yolov4/requirements.txt
  !pip install -qr koster_yolov4/yolov5_tracker/requirements.txt
  
  # Fix libmagic issue
  !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

  # Solution to avoid opencv and pims library issues based on https://stackoverflow.com/questions/71204741/how-to-fix-error-module-cv2-has-no-attribute-legacy-on-python-3-7-9-and-w
  #!pip uninstall -q opencv-python -y
  #!pip uninstall -q opencv-contrib-python -y
  #!pip install -q opencv-contrib-python

  # Replace upsampling script with custom version
  os.chdir("koster_yolov4/tutorials")
  !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

  # Replace nearest neighbours script with custom version (due to relative path issue)
  !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py

  # Enable external widgets
  from google.colab import output
  output.enable_custom_widget_manager()

  # Ensure widgets are shown properly
  !jupyter nbextension enable --user --py widgetsnbextension
  !jupyter nbextension enable --user --py jupyter_bbox_widget

  print("All packages are installed and ready to go!")
  try:
    clear_output()
    print("All packages are installed and ready to go!")
  except:
    clear_output()
    print("There have been some issues installing the packages!")
except:
  IN_COLAB = False
  import sys
  import pkgutil
  if pkgutil.find_loader('torch') is not None:
    !pip install -q --upgrade pip
    !pip install -q torch==1.8.0 torchvision==0.9.0
  # Ensure widgets are shown properly
  !jupyter nbextension enable --user --py widgetsnbextension
  !jupyter nbextension enable --user --py jupyter_bbox_widget
  print("Running locally... you're good to go!")

Running locally... you're good to go!


In [78]:
# Set the directory of the libraries
import sys, os
from pathlib import Path
#os.chdir("koster_yolov4/notebooks")
sys.path.append('..')

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from ipyfilechooser import FileChooser

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t6_utils as t6

# Model-specific imports
import yolov5.train as train
import yolov5.val as val
import yolov5.detect as detect

print("Packages loaded successfully")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

Packages loaded successfully


# 2. Evaluate model on custom footage

### Choose your project

In [81]:
!pip show torch

Name: torch
Version: 1.8.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /Users/jurie.germishuys/miniforge_x86_64/envs/odf/lib/python3.8/site-packages
Requires: numpy, typing-extensions
Required-by: thop, torchvision


In [19]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU', 'Medins', 'Test ML…

### Choose model

In [20]:
model = t6.choose_model(project_name.value)

Dropdown(description='Select model:', layout=Layout(width='50%'), options=(('fathomnet-benthic-yolov5', 'fatho…

Output()

In [87]:
download_dir = t_utils.choose_folder(".", "downloaded model")

FileChooser(path='.', filename='', title='HTML(value='Choose location of downloaded model')', show_hidden='Fal…

In [89]:
artifact_dir = t6.get_model(model_name = model.value,
                            project_name = project_name.value,
                            download_path=download_dir.value)

wandb: Downloading large artifact fathomnet-benthic-yolov5:latest, 175.84MB. 3 files... Done. 0:0:0
Checkpoint downloaded.


### Choose custom footage

In [83]:
source = t_utils.choose_folder(".", "custom footage")

FileChooser(path='.', filename='', title='HTML(value='Choose location of custom footage')', show_hidden='False…

### Choose where to save runs (this should be left as default value in most cases)

In [84]:
save_dir = t_utils.choose_folder(".", "runs output")

FileChooser(path='.', filename='', title='HTML(value='Choose location of runs output')', show_hidden='False', …

### Run model over selected custom footage

In [90]:
detect.run(weights=[f for f in Path(artifact_dir).iterdir() if f.is_file() and '.pt' in str(f)][0], 
           source=source.selected, conf_thres=0.5, save_txt=True, save_conf=True, half=True,
           project=save_dir.selected,
           name="detect")

YOLOv5 🚀 2022-10-10 Python-3.8.8 torch-1.8.0 CPU

Fusing layers... 
Model summary: 476 layers, 91841704 parameters, 0 gradients
image 1/103 /Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/data/images/hob16_11004 0 Lat 6277711.59 Lon 691999.61 Pan -129.9 Tilt 0.0 Hdg 51.1 17_08_15 17_08_560001_patch_0_4.jpg: 512x640 (no detections), 8547.1ms


KeyboardInterrupt: 

### View model output

In [33]:
eval_dir = t_utils.choose_folder(save_dir.selected if "save_dir" in vars() else '.', "runs output")

FileChooser(path='/Users/jurie.germishuys/Workspace/odf/koster-uw/models/yolov5/tutorials/detect3', filename='…

In [ ]:
t6.choose_files(eval_dir.selected)

### Investigate training and validation datasets

In [58]:
train_dataset, val_dataset = t6.get_dataset(project_name.value, model.value)

Externally trained model. No data available.


#### Training set

In [73]:
t6.get_data_viewer(os.path.join(train_dataset, "data/images"))

#### Validation set

In [75]:
t6.get_data_viewer(os.path.join(val_dataset, "data/images"))

# 3. Track unique individuals (optional)

In [ ]:

latest_tracker = t6.track_objects(source_dir=source.selected, 
                                  artifact_dir= artifact_dir,
                                  tracker_folder='../yolov5_tracker/',
                                  conf_thres=0.5,
                                  img_size=(720, 720),
                                  gpu=False)

### Generate classification report and counts by species

In [ ]:
t6.generate_csv_report(eval_dir.selected)

In [ ]:
t6.generate_counts(eval_dir.selected, latest_tracker, artifact_dir)

In [76]:
# END 